In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import datetime
import seaborn as sns

In [2]:
# Import local CSV files and change name of EXITS column

df_dec19 = pd.read_csv('v1december_turnstiles.csv', index_col = 0)
df_dec19 = df_dec19.rename(columns={'EXITS                                                               ':'EXITS'})
df_mar20 = pd.read_csv('v1march_turnstiles.csv', index_col = 0)
df_mar20 = df_mar20.rename(columns={'EXITS                                                               ':'EXITS'})
df_june20 = pd.read_csv('v1june_turnstiles.csv', index_col = 0)
df_june20 = df_june20.rename(columns={'EXITS                                                               ':'EXITS'})

In [3]:
mask_penn = (df_june20['STATION'] == '34 ST-PENN STA') & (df_june20['DATE'] == '06/06/2020')
df_june20[mask_penn].sample(20)

#df_june20.groupby(["C/A", "UNIT", "SCP","STATION","LINENAME"]).DATE.agg('sum')

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
55391,N071,R013,00-00-00,34 ST-PENN STA,ACE,IND,06/06/2020,20:00:00,REGULAR,6275509,17637258
55307,N070,R012,04-05-00,34 ST-PENN STA,ACE,IND,06/06/2020,20:00:00,REGULAR,117443241,0
137829,R139,R031,04-00-03,34 ST-PENN STA,123,IRT,06/06/2020,20:00:00,REGULAR,1554622,732305
137782,R139,R031,04-00-02,34 ST-PENN STA,123,IRT,06/06/2020,00:00:00,REGULAR,12984216,6765419
137702,R139,R031,04-00-00,34 ST-PENN STA,123,IRT,06/06/2020,16:00:00,REGULAR,930841,774350
54382,N067,R012,00-03-01,34 ST-PENN STA,ACE,IND,06/06/2020,16:00:00,REGULAR,5141060,1158508
137068,R138,R293,00-00-02,34 ST-PENN STA,123ACE,IRT,06/06/2020,02:00:00,REGULAR,23801037,12458331
54341,N067,R012,00-03-00,34 ST-PENN STA,ACE,IND,06/06/2020,20:00:00,REGULAR,1497265,801041
56187,N073,R013,02-00-02,34 ST-PENN STA,ACE,IND,06/06/2020,16:00:00,REGULAR,4030568,12038509
136903,R137,R031,02-03-00,34 ST-PENN STA,123,IRT,06/06/2020,12:00:00,REGULAR,201287,422897


In [4]:
df_june20[mask_penn]['LINENAME'].value_counts()

ACE       336
123ACE    138
123       126
Name: LINENAME, dtype: int64

In [5]:
new_time = df_dec19.TIME.map(lambda x: str(x)[:-6])
df_dec19['ABBREV_TIME'] = pd.to_numeric(new_time)

new_time = df_mar20.TIME.map(lambda x: str(x)[:-6])
df_mar20['ABBREV_TIME'] = pd.to_numeric(new_time)

new_time = df_june20.TIME.map(lambda x: str(x)[:-6])
df_june20['ABBREV_TIME'] = pd.to_numeric(new_time)

In [6]:
def twoPennStation(dataframe):
    dataframe.loc[(dataframe['STATION'] == '34 ST-PENN STA') & (dataframe['LINENAME'] == '123'), 'STATION'] = 'P34 ST-PENN EXPRESS'
    dataframe.loc[(dataframe['STATION'] == '34 ST-PENN STA') & (dataframe['LINENAME'] == 'ACE'), 'STATION'] = 'P34 ST-PENN LOCAL'
    
    # LINENAME 123ACE shares turnstiles with LINENAME ACE
    # Therefore, we add 123ACE into the ACE bin
    dataframe.loc[(dataframe['STATION'] == '34 ST-PENN STA') & (dataframe['LINENAME'] == '123ACE'), 'STATION'] = 'P34 ST-PENN EXPRESS'
    return dataframe
    
df_dec19 = twoPennStation(df_dec19)
df_mar20 = twoPennStation(df_mar20)
df_june20 = twoPennStation(df_june20)

In [7]:
print(len(df_dec19))
print(len(df_mar20))
print(len(df_june20))

618716
616472
620072


In [8]:
def hr4FootTraffic(dataframe):
    """
    This function calculated foot traffic for every 4 hour interval.
    It groups the data by TURNSTILE ("C/A", "UNIT", "SCP","STATION"), DATE, TIME, and TIMESTAMP,
    then aggregates it to show the sum (renamed as TOTAL_FOOTTRAFFIC).
    
    The last line for "station_counts_df" is used to calculate the max foot traffic by
    taking the sum of DEC19, MAR20, and JUNE20.
    """
    dataframe['TIMESTAMP'] = pd.to_datetime(dataframe['DATE'] + ' ' + dataframe['TIME'])
    dataframe['FOOTTRAFFIC'] = dataframe['ENTRIES'] + dataframe['EXITS']
    
    # This aggregates FOOTTRAFFIC by STATION & TIME, it also resets the indez
    hour4_counts_df = dataframe.groupby(["C/A", "UNIT", "SCP","STATION","DATE","TIME","TIMESTAMP",'ABBREV_TIME']).FOOTTRAFFIC.agg(['sum', 'count']).reset_index()

    # This creates a new column of DIFF_FOOTRAFFIC by taking the difference in 'sum'
    hour4_counts_df['DIFF_FOOTTRAFFIC'] = hour4_counts_df['sum'].diff()

    # For practical purposes, we will rename 'sum' to TOTAL_FOOTTRAFFIC
    hour4_counts_df = hour4_counts_df.rename(columns={'sum':'TOTAL_FOOTTRAFFIC'})

    # Now we'll drop out NaN values and reset the index
    hour4_counts_df = hour4_counts_df.dropna().reset_index(drop = True)

    #print(len(hour4_counts_df))
    
    # Below are steps for cleaning the data and removing outliers:
    
    # This removes all negative values in DIFF_FOOTTRAFFIC to ignore resets that occur
    hour4_counts_df = hour4_counts_df[hour4_counts_df['DIFF_FOOTTRAFFIC'] > 0]
    
    #print(len(hour4_counts_df))
    
    # This will create an upper bound of the 90th percentile, so we remove resets that result in DIFF high values
    hour4_counts_df = hour4_counts_df[hour4_counts_df['DIFF_FOOTTRAFFIC'] <=\
                                      np.percentile(hour4_counts_df['DIFF_FOOTTRAFFIC'], 90)]

    
    station_counts_df = hour4_counts_df.groupby(["STATION"], as_index=False)["DIFF_FOOTTRAFFIC"].sum()
    
    #print(len(hour4_counts_df))
    
    return hour4_counts_df, station_counts_df
    

df_dec19_4hrfoottraffic, df_dec19_maxfoottraffic  = hr4FootTraffic(df_dec19)
df_mar20_4hrfoottraffic, df_mar20_maxfoottraffic = hr4FootTraffic(df_mar20)
df_june20_4hrfoottraffic, df_june20_maxfoottraffic = hr4FootTraffic(df_june20)

618711
537543
483855
616470
530852
477833
620071
510146
459486


In [9]:
"""
This exports a CSV file that has contains:
(1) difference in foot traffic every 4 hours (DIFF_FOOTTRAFFIC)
(2) Removal of differences that result in negative values or those > 90th percentile (both outliers)
"""

"""
df_dec19_4hrfoottraffic.to_csv('v2_btn_december_turnstiles.csv')
df_mar20_4hrfoottraffic.to_csv('v2_btn_march_turnstiles.csv')
df_june20_4hrfoottraffic.to_csv('v2_btn_june_turnstiles.csv')
"""

print(len(df_dec19))
print(len(df_mar20))
print(len(df_june20))

618716
616472
620072


In [10]:
# Checking the top 20 most trafficked stations; the majority in top 10 match

total_foottraffic = df_dec19_maxfoottraffic.set_index('STATION').\
    join(df_mar20_maxfoottraffic.set_index('STATION'), lsuffix = '_DEC').\
    join(df_june20_maxfoottraffic.set_index('STATION'), lsuffix = '_MAR', rsuffix = '_JUNE')

total_foottraffic['DIFF_FOOTTRAFFIC_TOTAL'] = total_foottraffic['DIFF_FOOTTRAFFIC_DEC'] +\
    total_foottraffic['DIFF_FOOTTRAFFIC_JUNE'] + total_foottraffic['DIFF_FOOTTRAFFIC_MAR']

total_foottraffic = total_foottraffic.sort_values('DIFF_FOOTTRAFFIC_TOTAL', ascending=False).reset_index()

total_foottraffic_top20 = total_foottraffic.sort_values('DIFF_FOOTTRAFFIC_TOTAL', ascending=False).reset_index().head(20)

total_foottraffic_top20

,index,STATION,DIFF_FOOTTRAFFIC_DEC,DIFF_FOOTTRAFFIC_MAR,DIFF_FOOTTRAFFIC_JUNE,DIFF_FOOTTRAFFIC_TOTAL
0,0,FULTON ST,2098500.0,696579.0,376990.0,3172069.0
1,1,23 ST,2085516.0,655061.0,352235.0,3092812.0
2,2,86 ST,1807432.0,643391.0,323309.0,2774132.0
3,3,P34 ST-PENN LOCAL,1725435.0,521843.0,298240.0,2545518.0
4,4,GRD CNTRL-42 ST,1660989.0,651794.0,220905.0,2533688.0
5,5,59 ST,1637656.0,493547.0,242977.0,2374180.0
6,6,125 ST,1501259.0,572786.0,255378.0,2329423.0
7,7,34 ST-HERALD SQ,1482231.0,555760.0,282700.0,2320691.0
8,8,CANAL ST,1641083.0,448760.0,224831.0,2314674.0
9,9,42 ST-PORT AUTH,1577677.0,470966.0,150967.0,2199610.0


In [11]:
total_foottraffic.sort_values(['DIFF_FOOTTRAFFIC_TOTAL'], ascending = False).head(20);

In [12]:
# Top 10 Stations Plot

plt.figure(figsize=[12, 6])
font = {'family': 'sans-serif', 'weight': 'bold'}
plt.rc('font', **font)

ax1 = sns.barplot(x = 'STATION', y = total_foottraffic_top10['DIFF_FOOTTRAFFIC_TOTAL']/1e6,
                  data = total_foottraffic_top10,
                  palette = "muted",)
plt.xticks(rotation=20)
ax1.set_xlabel('Station Name', fontdict=font, fontsize='x-large')
ax1.set_ylabel('Daily Foot Traffic (Millions)', fontdict=font, fontsize='x-large')
plt.title('Top 10 Stations', fontdict=font, fontsize='xx-large');

NameError: name 'total_foottraffic_top10' is not defined

<Figure size 864x432 with 0 Axes>

In [ ]:
plt.figure(figsize=[12, 6])
font = {'family': 'sans-serif', 'weight': 'bold'}
plt.rc('font', **font)

# December 2019 data
ax1 = sns.lineplot(x = 'ABBREV_TIME', y = df_dec19_4hrfoottraffic['DIFF_FOOTTRAFFIC'],
                  data = df_dec19_4hrfoottraffic,
                  palette = "muted",)

# March 2020 data
sns.lineplot(x = 'ABBREV_TIME', y = df_mar20_4hrfoottraffic['DIFF_FOOTTRAFFIC'],
                     data = df_mar20_4hrfoottraffic,
                     palette = "muted", ax = ax1)

# June 2020 data
sns.lineplot(x = 'ABBREV_TIME', y = df_june20_4hrfoottraffic['DIFF_FOOTTRAFFIC'],
                     data = df_june20_4hrfoottraffic,
                     palette = "muted", ax = ax1)

ax1.set_xlabel('Military Time', fontdict=font, fontsize='x-large')
ax1.set_ylabel('Foot Traffic', fontdict=font, fontsize='x-large')

plt.legend(['December 2019', 'March 2020', 'June 2020'])
plt.xticks(rotation=0)
plt.xticks(np.arange(0, 25, 1.0))
plt.title('Amount of Riders/Month', fontdict=font, fontsize='x-large');

In [ ]:
justpenn = df_dec19[df_dec19["STATION"] == "34 ST-PENN STA"]
express = justpenn[justpenn["LINENAME"] == "123"]
print('***123***\n', express["TURNSTILE"].value_counts().head(3))
both = justpenn[justpenn["LINENAME"] == "123ACE"]
print('***123ACE***\n',express["TURNSTILE"].value_counts().head(3))
local = justpenn[justpenn["LINENAME"] == "ACE"]
print('***ACE***\n', local["TURNSTILE"].value_counts().head(3))

In [ ]:
df_june20[mask_penn].groupby(["C/A", "UNIT", "SCP","LINENAME"]).ENTRIES.agg(['sum']).to_csv('test1.csv')